# Tarea 1: Deep Learning

## Objetivos:
### El objetivo de esta tarea es poder demostrar el conocimiento adquirido en las clases 1-5, tutoriales 1-3 y el laboratorio formativo 1.

## Instrucciones:
- Para esta tarea debe completar las celdas faltantes, además de tomar decisiones sobre ciertos parámetros.
- Tendrán ya creada para ustedes un conjunto de entradas y salidas deseadas para una función, \( F(x,y) = x^2 + y^2 \), el objetivo principal es que dado un valor, la red sea capaz de predecir el resultado correcto con un umbral de precisión de x%, donde la nota será por puntos en relación a la precisión y error.

Por ejemplo, si de 100 predicciones la red creada tiene un 80% de precisión para |y_gorro - y_teoria| <= 0.1, tienes 15pts de 100, cuanto mayor sea la precisión, mayor la nota hasta 80/100 pts.

Los últimos 20 pts serán evaluados con base a las decisiones tomadas por ustedes, por esto en la última celda de tests es importante justificar las decisiones tomadas, sea la cantidad de capas ocultas, épocas, tasa de aprendizaje, etc.


### Formato de los datos
- Los datos se presentan con 2 valores de entrada y uno de salida: `input_1`, `input_2` y un valor de salida `output`.
- En total hay 1000 entradas (1000 `input_1`, 1000 `input_2`, 1000 `output`).


In [1]:
import numpy as np
import random


class Sigmoid:
    def derivative(self, x):
        return x * (1 - x)
    
    def __call__(self, x):
        return 1 / (1 + np.exp(-x))
    
class Linear:
    def derivative(self, x):
        return 1
    
    def __call__(self, x):
        return x

In [2]:
class Neuron:
    def __init__(self, weights, bias, activation = Sigmoid()):
        self.weights = weights
        self.bias = bias
        self.activation = activation
        self.output = 0
        self.inputs = []
        self.error = 0

    def feedforward(self, inputs):
        self.inputs = inputs
        total = np.dot(self.weights, inputs) + self.bias
        self.output = self.activation(total)
        return self.output
    
    def backpropagate_error(self, error):
        self.error = self.activation.derivative(self.output) * error

In [3]:
class NeuralNetwork:
    def __init__(self, input_red, output, h_layers, learning_rate=0.1):
        self.learning_rate = learning_rate
        self.layers = []
        self.input = input_red
        self.__init__layers(h_layers, output)
   

    
    def __init__layers(self, h_layers, output):
        for indice_layer in range(len(h_layers)):
            
                layer = []
                for cantidad_de_neuronas in range(h_layers[indice_layer]):
                    if indice_layer == 0:
                        neurona = Neuron(
                            [random.uniform(-1, 1) for _ in range(self.input)], random.uniform(-1, 1)
                        )
                    else:
                        neurona = Neuron(
                            [random.uniform(-1, 1) for _ in range(h_layers[indice_layer-1])], random.uniform(-1, 1)
                        )
                    layer.append(neurona)
                self.layers.append(layer)
                
        self.layers.append([Neuron([random.uniform(-1, 1) for _ in range(h_layers[-1])], random.uniform(-1, 1),Linear() ) for _ in range(output)])
    

    def feedforward(self, inputs):
        # your code here        
        for i_layer in range(len(self.layers)):
            outputs = []
            for i_neuron in range(len(self.layers[i_layer])):
                outputs.append(self.layers[i_layer][i_neuron].feedforward(inputs))
            inputs = outputs # para la proxima capa recibir lo que retorna la capa que acabamos de pasar
        return outputs
    
    def train(self, inputs, targets):
        outputs = self.feedforward(inputs)
        errors = [targets[i] - outputs[i] for i in range(len(outputs))]
    
        for indice_layers in range(len(self.layers)-1, -1, -1):
            layer = self.layers[indice_layers]
            next_layer = self.layers[indice_layers+1] if indice_layers != len(self.layers)-1 else None
            for indice_neurona in range(len(layer)):
                neuron = layer[indice_neurona]
                if next_layer is None:
                    neuron.backpropagate_error(errors[indice_neurona])
                else:
                    error = 0
                    for n in next_layer:
                        error += n.error * n.weights[indice_neurona]
                    neuron.backpropagate_error(error)    

        for indice_layers in range(len(self.layers)):
            for indice_neurona in range(len(self.layers[indice_layers])):
                
                neuron = self.layers[indice_layers][indice_neurona]
                # your code here
                for i_weight in range(len(neuron.weights)):
                    neuron.weights[i_weight] += self.learning_rate * neuron.error * neuron.inputs[i_weight]
                    neuron.bias += self.learning_rate * neuron.error
        error_final = sum([0.5*e**2 for e in errors]) / len(errors)  
        return error_final


In [132]:
import random
import numpy as np

# Generate random x, y values
random.seed("01101100 01100101 01101110 01101001 01101110")

# Generate training data for f(x, y) = x^2 + y^2
X = [[random.uniform(-1, 1), random.uniform(-1, 1)] for _ in range(1000)]
Y = [[x[0]**2 + x[1]**2] for x in X]

epocas = 1000

# El nombre de la rede debe ser rede_profunda
# your code here
rede_profunda = NeuralNetwork(2, 1, [5], learning_rate=0.1)

for i in range(epocas):
    error = 0
    for j in range(len(X)):
        # your code here
        error += rede_profunda.train(X[j], Y[j])
    if i % 100 == 0:
        print("error:", error)
print(error)

error: 242.3588546039383
error: 0.07301374743165766
error: 0.023544605072516135
error: 0.019484660011387676
error: 0.017717141034619115
error: 0.01642865975535478
error: 0.015390370276072842
error: 0.014524555986573304
error: 0.01378867116416565
error: 0.013154634840884768
0.012607429531411762


### Verificamos el valor predicho vs el valor real

In [134]:
random.seed(1)

# Generate training data for f(x, y) = x^2 + y^2
X = [[random.uniform(-1, 1), random.uniform(-1, 1)] for _ in range(1000)]
Y = [[x[0]**2 + x[1]**2] for x in X]


total_sample = len(X)
total_errors = 0
error_threshold = 0.05

for i in range(len(X)):
    error = np.abs(rede_profunda.feedforward(X[i])[0] - Y[i][0])
    if error > error_threshold:
        total_errors += 1

print(f'Errores mayor a {error_threshold}: {total_errors}')
print(f'Total muestra: {total_sample}')
print(f'PORCENTAJE ACIERTOS: {100 * (1 - (total_errors / total_sample))}')

Errores mayor a 0.05: 0
Total muestra: 1000
PORCENTAJE ACIERTOS: 100.0


Describe the task here!
# Describa todas las decisiones tomadas para tu solución
# Esto incluye la cantidad de épocas, tasa de aprendizaje, capas ocultas, capas de entrada y capas de salida.


Describe the task here!

En primer lugar, para validar que mi red estuviera bien entrenada, generé un set de pruebas que me permita validar que la red tenga una buena capacidad de generalización sobre datos desconocidos. Lo anterior lo realicé con este código:

```python
random.seed(1)

# Generate training data for f(x, y) = x^2 + y^2
X = [[random.uniform(-1, 1), random.uniform(-1, 1)] for _ in range(1000)]
Y = [[x[0]**2 + x[1]**2] for x in X]
```

Luego, las predicciones las realicé ejecutando el método `feedforward` de la red y seteando un nivel de tolerancia del 0.05 de acierto entre la predicción vs el valor real, con el siguiente código.

```python
total_sample = len(X)
total_errors = 0
error_threshold = 0.05

for i in range(len(X)):
    error = np.abs(rede_profunda.feedforward(X[i])[0] - Y[i][0])
    if error > error_threshold:
        total_errors += 1

print(f'Errores mayor a {error_threshold}: {total_errors}')
print(f'Total muestra: {total_sample}')
print(f'PORCENTAJE ACIERTOS: {100 * (1 - (total_errors / total_sample))}')
```

### Ajuste del Learning Rate

En segundo lugar, utilicé la configuración inicial de la red, con una capa oculta con 2 neuronas y un `learning_rate` de `0.1`. Para lo anterior, noté que el error se comenzaba a estancar en un valor alto en torno al valor 50, por lo que procedí a disminuirlo a `0.01`, lo cual mejoró el error, pero igual estancándose en torno a 44. Probé con los siguientes valores: `[1, 0.1, 0.01, 0.001, 0.0001]` y noté que con un valor alto como 1, tiende a oscilar el gradiente descendente, por lo que se torna ineficiente la red dado que habría que aumentar las epoch. Con esto consideré un `learning_rate=0.1` dado que puede hacer más eficiente el algoritmo en términos de rapidez de convergencia.

Con esta configuración obtuve un porcentaje de aciertos de `8.2 %`. Muy bajo aún.

### Ajuste de capas internas y neuronas

En tercer lugar, ya con `learning_rate=0.1`, probé la cantidad de neuronas en la capa interna con los siguientes valores: `[2,3,4,5,6,7]`. Para lo anterior obtuve 

    (1) NeuralNetwork(2, 1, [3], learning_rate=0.1) -> 92.0 % aciertos
    (2) NeuralNetwork(2, 1, [4], learning_rate=0.1) -> 96.7 % aciertos
    (3) NeuralNetwork(2, 1, [5], learning_rate=0.1) -> 100 % aciertos
    (4) NeuralNetwork(2, 1, [6], learning_rate=0.1) -> 100 % aciertos
    (5) NeuralNetwork(2, 1, [7], learning_rate=0.1) -> 100 % aciertos

Luego, probé con dos capas internas, por lo que dejé las siguientes configuraciones para `h_layers`: `[1,1], [2,1], [2,2], [2,3], [2,4], [3,1], [3,2], [3,3]`

    (6) NeuralNetwork(2, 1, [1,1], learning_rate=0.1) -> 8.6 % aciertos
    (7) NeuralNetwork(2, 1, [2,1], learning_rate=0.1) -> 9.09 % aciertos
    (8) NeuralNetwork(2, 1, [2,2], learning_rate=0.1) -> 9.19 % aciertos
    (9) NeuralNetwork(2, 1, [2,3], learning_rate=0.1) -> 84.2 % aciertos
    (10) NeuralNetwork(2, 1, [2,4], learning_rate=0.1) -> 18.4 % aciertos
    (11) NeuralNetwork(2, 1, [3,1], learning_rate=0.1) -> 98.5 % aciertos
    (12) NeuralNetwork(2, 1, [3,2], learning_rate=0.1) -> 98.5 % aciertos
    (13) NeuralNetwork(2, 1, [3,3], learning_rate=0.1) -> 99.0 % aciertos

Con lo anterior, vemos que los mejores resultados se obtienen con una sola capa interna, teniendo `5, 6` o `7` neuronas. Además, se puede ver que la incidencia de las neuronas de la primera capa es más positiva en cuanto a la predictibilidad, que en las neuronas de la segunda capa, por lo que podemos considerar que es mejor aumentar las neuronas de la primera capa. Por simplicidad y para prevenir overfitting, nos quedamos temporalmente con la configuración

    (3) NeuralNetwork(2, 1, [5], learning_rate=0.1) -> 100 % aciertos 

### Ajustes de las épocas

Ahora, solo en pos de buscar un modelo más sencillo que el que ya tenemos, compararemos la neurona elegida temporalmente `(3)` con la neurona `(11)`, la que tiene un alto porcentaje de aciertos pero quizás requiere más epochs. Para lo anterior utilizaremos 10.000 veremos cual tendrá los mejores resultados. Si la neurona `(11)` llega a tener 100% de aciertos, la elegiremos dado que tiene menos neuronas en total y podría ser una mejor alternativa a elegir.

    10.000 épocas: NeuralNetwork(2, 1, [5], learning_rate=0.1) -> 100 % aciertos
    10.000 épocas: NeuralNetwork(2, 1, [3,1], learning_rate=0.1) -> 98.5 % aciertos

Vimos que al cabo de la séptima iteración, el error ya se estancaba en torno al `0.273` en la red neuronal `(11)`, por lo que nos quedamos con la red neuronal

    NeuralNetwork(2, 1, [5], learning_rate=0.1) + 1.000 épocas

In [124]:
# Configuración ideal
# epocas = 1000
# rede_profunda = NeuralNetwork(2, 1, [5], learning_rate=0.1)